<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Account-Endpoints/blob/main/Query_Trading_Commission_Rate_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

查詢交易手續費率 \
取得當前使用者的交易費率

GET/openApi/swap/v2/user/commissionRate \
頻率限制：依UID限制為每秒5次 & 依IP組合限制數量 \
API KEY權限：永續合約交易

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 填入你的 API Key
SECRETKEY = ""  # 填入你的 Secret Key

In [ ]:
def demo():
    payload = {}  # GET 請求無主體
    path = '/openApi/swap/v2/user/commissionRate'  # 查詢交易手續費率 API 路徑
    method = "GET"  # HTTP 方法

    paramsMap = {
        "recvWindow": "5000"  # 請求有效時間窗口，單位毫秒
    }

    paramsStr = parseParam(paramsMap)  # 將參數轉換為 URL 查詢字串並加入 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 使用 HMAC-SHA256 演算法產生簽名，確保請求完整與安全
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 印出簽名以便除錯
    return signature

In [ ]:
def send_request(method, path, urlpa, payload):
    # 組合完整請求 URL，並加入簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出請求 URL 方便檢查

    headers = {
        'X-BX-APIKEY': APIKEY,  # 在標頭帶入 API Key
    }

    response = requests.request(method, url, headers=headers, data=payload)  # 發送 HTTP 請求
    return response.text  # 回傳 API 回應字串

In [ ]:
def parseParam(paramsMap):
    # 將參數依鍵名排序並組合成 URL 查詢字串
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    # 加入毫秒級 timestamp 防止重放攻擊
    if paramsStr != "":
        return paramsStr + "&timestamp=" + str(int(time.time() * 1000))
    else:
        return "timestamp=" + str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行範例並印出結果